In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Import custom module
import sys
import os
sys.path.insert(0,os.path.abspath("../scripts"))
import modeling as mo
import importlib
importlib.reload(mo)

<module 'modeling' from 'c:\\Users\\HP\\Desktop\\TenX\\insurance_data_analysis\\scripts\\modeling.py'>

In [21]:

data = pd.read_csv('../data/cleaned_insurance_data.csv')

C:\Users\HP\AppData\Local\Temp\ipykernel_20620\3416413044.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/cleaned_insurance_data.csv')


In [22]:
# Data Preprocessing
df = mo.handle_missing_data(data)  # Handle missing values
df = mo.feature_engineering(df)  # Feature engineering
df = mo.process_dates(df) # Date Formatting
df.head()


,UnderwrittenCoverID,PolicyID,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,MaritalStatus,...,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,VehicleAge,ClaimsToPremiumRatio,TransactionYear,TransactionMonthOnly
0,145249,12827,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,19,0.0,2015,3
1,145249,12827,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,19,0.0,2015,5
2,145249,12827,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,19,0.0,2015,7
3,145255,12827,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,19,0.0,2015,5
4,145255,12827,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,19,0.0,2015,7


In [23]:

# Encode categorical variables
categorical_columns = ['Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType',
                       'MaritalStatus', 'Gender', 'Country', 'Province', 'VehicleType', 'ItemType']

df_encoded = mo.encode_categorical(df, categorical_columns)

# Check the encoded DataFrame
print(df_encoded.head())


Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, IsVATRegistered, PostalCode, MainCrestaZone, SubCrestaZone, mmcode, RegistrationYear, make, Model, Cylinders, cubiccapacity, kilowatts, bodytype, NumberOfDoors, VehicleIntroDate, CustomValueEstimate, AlarmImmobiliser, TrackingDevice, CapitalOutstanding, NewVehicle, WrittenOff, Rebuilt, Converted, CrossBorder, NumberOfVehiclesInFleet, SumInsured, TermFrequency, CalculatedPremiumPerTerm, ExcessSelected, CoverCategory, CoverType, CoverGroup, Section, Product, StatutoryClass, StatutoryRiskType, TotalPremium, TotalClaims, VehicleAge, ClaimsToPremiumRatio, TransactionYear, TransactionMonthOnly, Citizenship_AF, Citizenship_ZA, Citizenship_ZW, LegalType_Individual, LegalType_Partnership, LegalType_Private company, LegalType_Public company, LegalType_Sole proprieter, Title_Miss, Title_Mr, Title_Mrs, Title_Ms, Bank_Capitec Bank, Bank_First National Bank, Bank_FirstRand Bank, Bank_Investec Bank, Bank_Ithala Bank, Bank_Mercantile Lisbon Bank

In [24]:
# Check for non-numeric columns
non_numeric_columns = df.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_columns)

Non-numeric columns: Index(['Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType',
       'MaritalStatus', 'Gender', 'Country', 'Province', 'MainCrestaZone',
       'SubCrestaZone', 'ItemType', 'VehicleType', 'make', 'Model', 'bodytype',
       'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt',
       'Converted', 'CrossBorder', 'TermFrequency', 'ExcessSelected',
       'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product',
       'StatutoryClass', 'StatutoryRiskType'],
      dtype='object')


Train test split

In [32]:
""" # Train-Test Split
X = df.drop(['TotalPremium', 'TotalClaims'], axis=1)  # Features
y = df['TotalPremium']  # Target (for example, predicting TotalPremium)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.dtypes) """

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler



# Encoding Functions
def encoder(method, dataframe, columns_label, columns_onehot):
    if method == 'labelEncoder':      
        df_lbl = dataframe.copy()
        for col in columns_label:
            label = LabelEncoder()
            df_lbl[col] = label.fit_transform(df_lbl[col].astype(str))
        return df_lbl
    
    elif method == 'oneHotEncoder':
        df_oh = dataframe.copy()
        df_oh = pd.get_dummies(data=df_oh, columns=columns_onehot, drop_first=True)
        return df_oh

# Define columns
columns_label = ['LegalType']
columns_onehot = ['Citizenship', 'Country', 'VehicleType']

# Apply label encoding
df_encoded_label = encoder('labelEncoder', df, columns_label, [])

# Apply one-hot encoding
df_encoded = encoder('oneHotEncoder', df_encoded_label, [], columns_onehot)

# Prepare features and target
X = df_encoded.drop('TotalPremium', axis=1)
y = df_encoded['TotalPremium']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features (if any)
scaler = StandardScaler()
numerical_columns = ['RegistrationYear']  # Include other numerical columns if present
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

# Train Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)
print("Predictions:", y_pred)



Predictions: [1500.]


Modeling

Linear Regression model

In [33]:
linear_model = mo.train_linear_regression(X_train, y_train)
y_pred_lr = linear_model.predict(X_test)

Random Forest model

In [34]:
rf_model = mo.train_random_forest(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

Train an XGBoost model

In [35]:
xgb_model = mo.train_xgboost(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

Model Evaluation

In [36]:
lr_results = mo.evaluate_model(y_test, y_pred_lr)
rf_results = mo.evaluate_model(y_test, y_pred_rf)
xgb_results = mo.evaluate_model(y_test, y_pred_xgb)

c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [37]:
# Print the results
print("Linear Regression Results:", lr_results)
print("Random Forest Results:", rf_results)
print("XGBoost Results:", xgb_results)

Linear Regression Results: {'MSE': 90000.0, 'MAE': 300.0, 'R2': nan}
Random Forest Results: {'MSE': 178084.0, 'MAE': 422.0, 'R2': nan}
XGBoost Results: {'MSE': 249999.02343845367, 'MAE': 499.9990234375, 'R2': nan}


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import shap
import lime
import lime.lime_tabular

# Random Forest Feature Importance
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importance - Random Forest")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.tight_layout()
plt.show()
